<a href="https://colab.research.google.com/github/OmGaler/MSK-Capstone/blob/master/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup for nnUNet

In [2]:
import os
# Set environment variable
os.environ['nnUNet_raw'] = 'nnUNet_raw'
os.environ['nnUNet_preprocessed'] = 'nnUNet_preprocessed'
os.environ['nnUNet_results'] = 'nnUNet_results'

In [2]:
!pip install torchio

  Using cached torchio-0.19.6-py2.py3-none-any.whl (173 kB)
  Using cached humanize-4.9.0-py3-none-any.whl (126 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
  Using cached rich-13.7.1-py3-none-any.whl (240 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)


In [3]:
! mkdir 'nnUNet_raw'
! mkdir 'nnUNet_raw/Dataset007_Kidney'
! mkdir 'nnUNet_raw/Dataset007_Kidney/imagesTs'
! mkdir 'nnUNet_raw/Dataset007_Kidney/imagesTr'
! mkdir 'nnUNet_raw/Dataset007_Kidney/labelsTr'

mkdir: cannot create directory ‘nnUNet_raw’: File exists
mkdir: cannot create directory ‘nnUNet_raw/Dataset007_Kidney’: File exists
mkdir: cannot create directory ‘nnUNet_raw/Dataset007_Kidney/imagesTs’: File exists
mkdir: cannot create directory ‘nnUNet_raw/Dataset007_Kidney/imagesTr’: File exists
mkdir: cannot create directory ‘nnUNet_raw/Dataset007_Kidney/labelsTr’: File exists


In [4]:
!git clone https://github.com/MIC-DKFZ/batchgenerators.git
%cd batchgenerators
!pip install .

%cd ..


fatal: destination path 'batchgenerators' already exists and is not an empty directory.
/mnt/c/Users/omerg/OneDrive - Yeshiva University/MSK/MSK-Capstone/batchgenerators
Processing /mnt/c/Users/omerg/OneDrive - Yeshiva University/MSK/MSK-Capstone/batchgenerators
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Created wheel for batchgenerators: filename=batchgenerators-0.25-py3-none-any.whl size=93206 sha256=1e4abc306d20df4c916b2be8ce76a03c5c424d05bbf96c2d42b9fd41affc201b
  Stored in directory: /tmp/pip-ephem-wheel-cache-bo84v8du/wheels/65/f4/e7/19569f671f3f59132a621680a96dd9ba6ae33df4381fad752f
Successfully built batchgenerators
  Attempting uninstall: batchgenerators
    Found existing installation: batchgenerators 0.25
    Uninstalling batchgenerators-0.25:
      Successfully uninstalled batchgenerators-0.25
/mnt/c/Users/omerg/OneDrive - Yeshiva University/MSK/MSK-Capstone


In [5]:
! git clone https://github.com/OmGaler/nnUNet.git
! pip install -e nnUNet/

fatal: destination path 'nnUNet' already exists and is not an empty directory.
Obtaining file:///mnt/c/Users/omerg/OneDrive%20-%20Yeshiva%20University/MSK/MSK-Capstone/nnUNet
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Running setup.py develop for nnunetv2


Download the data

In [6]:
!pip install --upgrade --no-cache-dir gdown
!gdown --id "1ZZ5X4y4Ywyv8GCm7td0TEGVxZOib2xXQ"

     |████████████████████████████████| 147 kB 549 kB/s eta 0:00:01
/home/omer/.local/lib/python3.9/site-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1ZZ5X4y4Ywyv8GCm7td0TEGVxZOib2xXQ

but Gdown can't. Please check connections and permissions.


In [7]:
!unzip inverse_300.zip -d inverse_300

Archive:  inverse_300.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
note:  inverse_300.zip may be a plain executable, not an archive
unzip:  cannot find zipfile directory in one of inverse_300.zip or
        inverse_300.zip.zip, and cannot find inverse_300.zip.ZIP, period.


Round labels (they may be rounded already, but what's the harm?)

In [9]:
import sys
import os
import shutil
import numpy as np
import nibabel as nib

def round_labels(input_dir, output_dir):
    # Iterate over files in the input directory
    os.mkdir(output_dir)
    for folder in os.listdir(input_dir):
        if folder[:4]== 'case':
            # if filename.endswith(".nii.gz"):
            folderpath = os.path.join(input_dir,   folder)
            os.makedirs(os.path.join(output_dir,folder))
            for filename in os.listdir(folderpath):
                input_file = os.path.join(input_dir,   folder, filename)
                output_file = os.path.join(output_dir, folder, filename)
                if filename == "segmentation.nii.gz":
                    # Load the segmentation image
                    img = nib.load(input_file)
                    data = img.get_fdata()

                    # Round the labels
                    rounded_data = np.rint(data).astype(np.uint8)

                    # Save the rounded labels to a new NIfTI file
                    rounded_img = nib.Nifti1Image(rounded_data, img.affine, img.header)
                    nib.save(rounded_img, output_file)
                else:
                    shutil.copyfile(input_file, output_file)
        else:
            print("Didn't enter folder",folder)


round_labels("inverse_300", "inverse_300_rounded")

Remove the third label

In [10]:
import nibabel as nib
import os


def modify_segmentation_file(seg_file_path):
    # load the image
    img = nib.load(seg_file_path)
    # get the image data
    data = img.get_fdata()
    # change any pixel with a value of 3 to a 1
    data[data == 3] = 1
    # create the new image
    modified_img = nib.Nifti1Image(data, img.affine, img.header)
    # delete the old file
    os.remove(seg_file_path)
    print("Removed " + seg_file_path)
    # save the image
    nib.save(modified_img, seg_file_path)
    print("saved new " + seg_file_path)


cases_dir = '/inverse_300_rounded'
# Iterate over all folders in cases_dir
for root, dirs, files in os.walk(cases_dir):
    for file in files:
        if file == "segmentation.nii.gz":
            seg_file_path = os.path.join(root, file)
            modify_segmentation_file(seg_file_path)



In [11]:
import os
import shutil

# Define the paths to the source and destination folders
source_folder = "inverse_300_rounded"
destination_train_folder = "nnUNet_raw/Dataset007_Kidney/imagesTs"
destination_test_folder = "nnUNet_raw/Dataset007_Kidney/labelsTs"

# Create destination folders if they don't exist
os.makedirs(destination_train_folder, exist_ok=True)
os.makedirs(destination_test_folder, exist_ok=True)

# Iterate through each folder in the source directory
for case_folder in os.listdir(source_folder):
    case_folder_path = os.path.join(source_folder, case_folder)

    # Check if the item in the source folder is a directory
    if os.path.isdir(case_folder_path):
        # Find the imaging.nii.gz and segmentation.nii.gz files
        imaging_file = os.path.join(case_folder_path, "imaging.nii.gz")
        segmentation_file = os.path.join(case_folder_path, "segmentation.nii.gz")

        # Check if both files exist
        if os.path.exists(imaging_file) and os.path.exists(segmentation_file):
            # Rename and move the imaging file to the training folder
            new_imaging_file = os.path.join(destination_train_folder, f"{case_folder}_0000.nii.gz")
            try:
                shutil.move(imaging_file, new_imaging_file)
                print(f"Imaging file for {case_folder} moved successfully.")
            except FileNotFoundError:
                print(f"Error: Imaging file not found for {case_folder}. Skipping...")

            # Rename and move the segmentation file to the testing folder
            new_segmentation_file = os.path.join(destination_test_folder, f"{case_folder}.nii.gz")
            try:
                shutil.move(segmentation_file, new_segmentation_file)
                print(f"Segmentation file for {case_folder} moved successfully.")
            except FileNotFoundError:
                print(f"Error: Segmentation file not found for {case_folder}. Skipping...")
        else:
            print(f"Files not found in {case_folder}. Skipping...")


Imaging file for case_00004 moved successfully.
Segmentation file for case_00004 moved successfully.
Imaging file for case_00006 moved successfully.
Segmentation file for case_00006 moved successfully.
Imaging file for case_00011 moved successfully.
Segmentation file for case_00011 moved successfully.
Imaging file for case_00013 moved successfully.


Segmentation file for case_00013 moved successfully.
Imaging file for case_00016 moved successfully.
Segmentation file for case_00016 moved successfully.
Imaging file for case_00022 moved successfully.
Segmentation file for case_00022 moved successfully.
Imaging file for case_00023 moved successfully.
Segmentation file for case_00023 moved successfully.
Imaging file for case_00028 moved successfully.
Segmentation file for case_00028 moved successfully.
Imaging file for case_00029 moved successfully.
Segmentation file for case_00029 moved successfully.
Imaging file for case_00034 moved successfully.
Segmentation file for case_00034 moved successfully.
Imaging file for case_00038 moved successfully.
Segmentation file for case_00038 moved successfully.
Imaging file for case_00044 moved successfully.
Segmentation file for case_00044 moved successfully.
Imaging file for case_00045 moved successfully.
Segmentation file for case_00045 moved successfully.
Imaging file for case_00047 moved succ

In [12]:
! mkdir output_folder

mkdir: cannot create directory ‘output_folder’: File exists


In [13]:
! mkdir -p nnUNet_results/Dataset007_Kidney/nnUNetTrainer__nnUNetPlans__2d

In [14]:
import json

# Define the dataset information
dataset_info = {
    "channel_names": {
        "0": "CT"
    },
    "labels": {
        "background": 0,
        "kidney": [1, 2],
        "tumor": 2
    },
    "numTraining": 300,
    "file_ending": ".nii.gz",
    "overwrite_image_reader_writer": "NibabelIOWithReorient",
    "regions_class_order": [1, 2]
}

# Define the file path for saving the JSON file
output_file = 'nnUNet_results/Dataset007_Kidney/nnUNetTrainer__nnUNetPlans__2d/dataset.json'

# Write the dataset information to the JSON file
with open(output_file, 'w') as json_file:
    json.dump(dataset_info, json_file, indent=4)


In [5]:
!nnUNetv2_predict -i "nnUNet_raw/Dataset007_Kidney/imagesTs" -o "output_folder" -d 7 -c 2d -chk "checkpoint_best.pth" -f 2 #-device cpu


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 189 cases in the source folder
I am process 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 189 cases that I would like to predict

Predicting case_00004:
perform_everything_on_device: True
100%|█████████████████████████████████████████| 128/128 [00:31<00:00,  4.08it/s]
sending off prediction to background worker for resampling and export
done with case_00004

Predicting case_00006:
perform_everything_on_device: True
100%|█████████████████████████████████████████| 314/314 [01:13<00:00,  4.25it/s]
sending off prediction to background worker for resampling and 

In [4]:
!zip -r output_folder.zip output_folder/

  adding: output_folder/ (stored 0%)
  adding: output_folder/case_00004.nii.gz (deflated 79%)
  adding: output_folder/case_00006.nii.gz (deflated 70%)
  adding: output_folder/case_00011.nii.gz (deflated 66%)
  adding: output_folder/case_00013.nii.gz (deflated 71%)
  adding: output_folder/case_00016.nii.gz (deflated 69%)
  adding: output_folder/case_00022.nii.gz (deflated 71%)
  adding: output_folder/case_00023.nii.gz (deflated 70%)
  adding: output_folder/case_00028.nii.gz (deflated 63%)
  adding: output_folder/case_00029.nii.gz (deflated 75%)
  adding: output_folder/case_00034.nii.gz (deflated 70%)
  adding: output_folder/case_00038.nii.gz (deflated 65%)
  adding: output_folder/case_00044.nii.gz (deflated 74%)
  adding: output_folder/case_00045.nii.gz (deflated 69%)
  adding: output_folder/case_00047.nii.gz (deflated 75%)
  adding: output_folder/case_00048.nii.gz (deflated 81%)
  adding: output_folder/case_00049.nii.gz (deflated 71%)
  adding: output_folder/case_00050.nii.gz (deflated